In [1]:
from bs4 import BeautifulSoup
import json
import csv
import requests
import random
import re
import pandas as pd

Getting the institue links, we need those to scrape info about schools.

In [15]:
institute_link = []
link_ids = {}

with open("data/infoscuola-database.json","r") as f:
    file_data = json.load(f)
    for school_info in file_data['data']:

        institute_name = school_info[0].split('"')
        link_info = institute_name[1].split("/")
        id_link = link_info[-1]
        
        if(id_link not in link_ids):
            institute_link.append(institute_name[1])
            link_ids[id_link] = True
        

Getting school links

In [58]:
# id to look for in the HTML: table-unita-scolastiche
schools_link = []

for link in institute_link:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")
    table = soup.find("table",attrs={'id':'table-unita-scolastiche'})
    tbody = table.find("tbody")
    a_tags = tbody.findAll(["a"],href=True)

    for a_tag in a_tags:
        schools_link.append("https://www.vivoscuola.it"+a_tag['href'])
    random.uniform(0.9,1.2)
    

In [59]:
with open('data/schoo_links.txt', 'w') as f:
    for line in schools_link:
        f.write(f"{line}\n")


We now parse school details.

In [13]:
schools_link = []
with open("data/schoo_links.txt","r") as f:
    schools_link = f.readlines()

schools_details = []

for link in schools_link:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")

    entry = [None,None,None,None,None,None,None,None,None,None,None,None]
    school_info = {
        "Tipo scuola":[0,None],
        "Indirizzo":[0,None],
        "Telefono":[0,None],
        "Fax":[0,None],
        "Email istituto":[0,None],
        "Email dirigenza":[0,None],
        "Email segreteria":[0,None],
        "Sito web":[0,None],
        "Codice MIUR":[0,None]
    }
    title = soup.find("div",attrs={'class':'u-content-title'})
    entry[9] = title.text
    entry[10] = link.split("/")[-1]
            
    main_div = soup.find("div",attrs={'class':"Grid Prose u-padding-top-xxl"})
        
    info_descriptions = main_div.select("div[class*=u-textItalic]")

    for index,info in enumerate(info_descriptions):
        school_info[info.text.strip()] = [index,info.text.strip()]
    general_infos = main_div.select("div[class*=u-size3of4]")

    for index,key in enumerate(school_info):
        if(school_info[key][1] != None):
            entry[index] = general_infos[school_info[key][0]].text.strip()

    div_info_istituto = soup.select_one("div.Grid.Prose.u-padding-bottom-s")

    divs = div_info_istituto.find_all("div")

    for i in range(len(divs)):
        if "Scuola" in divs[i].getText():
            a = divs[i+1].find("a")

            entry[11] = a['href']
            break
    
    
    schools_details.append(entry)
    random.uniform(0.9,1.4)
    
#scripts = soup.find_all("script")
#pattern = re.compile('var provincecode (.*?)',re.MULTILINE)
#for script in scripts:
#    print(script)
#    if(pattern.match(str(script.string))):
#        data = pattern.match(script.string)
#        print(data)
#    break

    

In [11]:
schools_details

[['Provinciale',
  'Piazza Municipio,\n    \n5 -\n    \n38050    \nNOVALEDO',
  '+39 0461 764581',
  '+39 0461 771046',
  'ic.centrovalsugana@pec.provincia.tn.it',
  'dir.ic.centrovalsugana@scuole.provincia.tn.it',
  'segr.ic.centrovalsugana@scuole.provincia.tn.it',
  'http://www.iccentrovalsugana.it',
  'TNEE813064',
  '\n\n                \nSCUOLA PRIMARIA "C. CORRADI" NOVALEDO\n            \n',
  '29503\n',
  '\n\n\n                        ISTITUTO COMPRENSIVO CENTRO VALSUGANA\n                    \n\n'],
 ['Provinciale',
  'Via Baldessari,\n    \n15 -\n    \n38050    \nRONCEGNO TERME',
  '+39 0461 764581',
  '+39 0461 771046',
  'ic.centrovalsugana@pec.provincia.tn.it',
  'dir.ic.centrovalsugana@scuole.provincia.tn.it',
  'segr.ic.centrovalsugana@scuole.provincia.tn.it',
  'http://www.iccentrovalsugana.it',
  'TNEE813075',
  '\n\n                \nSCUOLA PRIMARIA "P. MARTINELLI" RONCEGNO TERME\n            \n',
  '29504\n',
  '\n\n\n                        ISTITUTO COMPRENSIVO CENT

In [14]:
with open("data/school_list_cleaned.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["school_type","address","phone_number","fax_number","school_email","management_email","office_email","website","miur_code","school_name","school_id","institute_id"])
    writer.writerows(schools_details)

fetching institutes info

In [16]:
institutes_details = []

institutes_dropouts = {}

institutes_fetched = {}

for school_detail in schools_details:

    if(school_detail[-1] in institutes_fetched):
        continue 
    response = requests.get("https://www.vivoscuola.it"+school_detail[-1])
    soup = BeautifulSoup(response.content,"html.parser")

    entry = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]
    institute_info = {
        "Coordinatore pedagogico":[0,None],
        "Presidente Ente Gestore":[0,None],
        "Sorastant de la scola ladines":[0,None],
        "Tipo istituto":[0,None],
        "Dirigente scolastico":[0,None],
        "Direttore":[0,None],
        "Indirizzo":[0,None],
        "Telefono":[0,None],
        "Fax":[0,None],
        "Email istituto":[0,None],
        "Email dirigenza":[0,None],
        "Email segreteria":[0,None],
        "Sito web":[0,None],
        "Codice MIUR":[0,None],
        "Numero plessi/scuole":[0,None]
    }    
    title = soup.find("div",attrs={'class':'u-content-title'})
    entry[15] = title.text
    entry[16] = school_detail[-1]
            
    main_div = soup.find("div",attrs={'class':"Grid Prose u-padding-top-xxl"})
        
    info_descriptions = main_div.select("div[class*=u-textItalic]")

    for index,info in enumerate(info_descriptions):
        institute_info[info.text.strip()] = [index,info.text.strip()]

    general_infos = main_div.select("div[class*=u-size3of4]")

    for index,key in enumerate(institute_info):
        if(institute_info[key][1] != None):
            try:
                entry[index] = general_infos[institute_info[key][0]].text.strip()
            except IndexError:
                print(f"ERRORE: {link}")
        #aprilascuol
    #institutes_dropouts[institute_info["Codice MIUR"]] = [] #nome, [ordine,tipo,abbandoni,iscritti]

    aprilascuola = soup.find("div",attrs={'class':'aprilascuolaaffix'})
    dropdown = aprilascuola.find("div", attrs={'class':'dropdown'})
    menu = dropdown.find("div",attrs={'class':"dropdown-menu"})
    list_items = menu.find_all("li")
    for item in list_items:
        span = item.find("span")
        if("Abbandono" in span.text):
            a_tag = item.find("a")
            institutes_dropouts[entry[13]] = []
            institutes_dropouts[entry[13]].append("https://www.vivoscuola.it"+a_tag['href'])
    
    
    institutes_details.append(entry)

    institutes_fetched[school_detail[-1]] = True
    random.uniform(0.9,1.4)

In [17]:

with open("data/institutes_list.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["coordinatore_pedagogico",
    "presidente_ente_gestore",
    "sorastant",
    "tipo_instituto",
    "dirigente_scolastico",
    "direttore",
    "indirizzo",
    "telefono",
    "fax",
    "email_istituto",
    "email_dirigenza",
    "email_segreteria",
    "sito_web",
    "miur_code",
    "numero_plessi_scuole",
    "nome_istituto",
    "id_istituto"])
    writer.writerows(institutes_details)

we now parse dropouts rate using information gained in the previous step

In [ ]:

api_code_per_link = []
for key in institutes_dropouts:
    link = institutes_dropouts[key][0]
    print(link)
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")

    scripts  = soup.find_all("script",{"scr":False})

    p = re.compile('var provincecode = (.*);')        
    for script in scripts:
        if("provincecode" in script.text):
            match = p.search(script.string)
            api_code = match.group(1)
            if(api_code[0] == '"'):
                api_code = api_code[1:-1]
            api_code_per_link.append((key,link,api_code))
    random.uniform(0.9,1.4)


In [ ]:
api_code_per_link

In [29]:
endpoint = "https://www.istruzione.provincia.tn.it/services/sei/api/v1/institutes/schoolDropout/"
institutes_dropouts_data = []

def check_for_key_error(dictionary,key):
    if(key not in dictionary):
        return None
    else:
        return dictionary[key]

for institute_api_infos in api_code_per_link:
    miur_code,link,api_code = institute_api_infos
    response = requests.get(url=endpoint+api_code)
    data = response.json()

    for entry in data['listaDati']:
        #dataInserimento
        #ordScolCap -> tipo scuola
        #ordScolOrd
        #struttLiv1Cap
        #struttLiv1Miur
        #struttLiv1CodPat
        #tipoOraId
        #valore -> abbandoni
        #valoreD -> iscritti
        #valoreN -> boh
        #miurocode del ciclo (per vedere se combaciano),
        if(check_for_key_error(entry,"ordScolCap")== None or "Totale" in check_for_key_error(entry,"ordScolCap")):
            continue

        institutes_dropouts_data.append([
            check_for_key_error(entry,'dataInserimento'),
            check_for_key_error(entry,'ordScolCap'),
            check_for_key_error(entry,'ordScolOrd'),
            check_for_key_error(entry,'struttLiv1Cap'),
            check_for_key_error(entry,'struttLiv1CodMiur'),
            check_for_key_error(entry,'struttLiv1CodPat'),
            check_for_key_error(entry,'tipoOraId'),
            check_for_key_error(entry,'valore'),
            check_for_key_error(entry,'valoreD'),
            check_for_key_error(entry,'valoreN'),
            check_for_key_error(entry,"annoScolCap"),
            miur_code
        ])

    random.uniform(.9,1.2)

with open("data/institute_droputs.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["data_inserimento","ord_scol_cap","ord_scol_ord","strutt_liv_1_cap","strutt_liv_1_cod_miur","strutt_liv_1_cod_pat","tipo_ora_id","valore","valore_d","valore_n","anno_scol","miur_code"])
    writer.writerows(institutes_dropouts_data)


fetching now students who failed class

In [30]:
endpoint = "https://www.istruzione.provincia.tn.it/services/sei/api/v1/institutes/schoolResults/"
institutes_ripetenti_data = []

for institute_api_infos in api_code_per_link:
    miur_code,link,api_code = institute_api_infos
    response = requests.get(url=endpoint+api_code)
    data = response.json()

    for entry in data['listaDati']:
        #dataInserimento
        #ordScolCap -> tipo scuola
        #ordScolOrd
        #struttLiv1Cap
        #struttLiv1Miur
        #struttLiv1CodPat
        #tipoOraId
        #valore -> abbandoni
        #valoreD -> iscritti
        #valoreN -> boh
        #miurocode del ciclo (per vedere se combaciano),
        #annoCorsAggCap -> anno corso (prima, seconda, terza..)
        if(check_for_key_error(entry,"ordScolCap")== None or "Totale" in check_for_key_error(entry,"ordScolCap")):
            continue

        institutes_ripetenti_data.append([
            check_for_key_error(entry,'dataInserimento'),
            check_for_key_error(entry,'ordScolCap'),
            check_for_key_error(entry,'ordScolOrd'),
            check_for_key_error(entry,'struttLiv1Cap'),
            check_for_key_error(entry,'struttLiv1CodMiur'),
            check_for_key_error(entry,'struttLiv1CodPat'),
            check_for_key_error(entry,'tipoOraId'),
            check_for_key_error(entry,'valore'),
            check_for_key_error(entry,'valoreD'),
            check_for_key_error(entry,'valoreN'),
            check_for_key_error(entry,"annoScolCap"),
            check_for_key_error(entry,"annoCorsAggCap"),
            miur_code
        ])

    random.uniform(.9,1.2)

with open("data/institute_ripetenti.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["data_inserimento","ord_scol_cap","ord_scol_ord","strutt_liv_1_cap","strutt_liv_1_cod_miur","strutt_liv_1_cod_pat","tipo_ora_id","valore","valore_d","valore_n","anno_scol","anno_corso","miur_code"])
    writer.writerows(institutes_ripetenti_data)


fetching now grading results

In [34]:
endpoint = "https://www.istruzione.provincia.tn.it/services/sei/api/v1/institutes/gradingResults/"
institutes_ripetenti_data = []

for institute_api_infos in api_code_per_link:
    miur_code,link,api_code = institute_api_infos
    response = requests.get(url=endpoint+api_code)
    data = response.json()

    for d in data:
        ord_scol = check_for_key_error(d,"ordineScuola")
        tipo_orario = check_for_key_error(d,"tipoOrario")
        numero_anni = check_for_key_error(d,"numeroAnni")

        esitiScrutini = check_for_key_error(d,"esitiScrutini")
        for esito in esitiScrutini:
            etichetta = check_for_key_error(esito,"etichetta")

            if("PAT" not in etichetta):
                percentuali = check_for_key_error(esito,"percentuali")
                
                for index,percentuale in enumerate(percentuali):
                    #years
                    institutes_ripetenti_data.append([
                        ord_scol,
                        tipo_orario,
                        numero_anni,
                        etichetta,
                        percentuale,
                        index+1, #anno
                        miur_code]
                    )
    random.uniform(.9,1.2)

with open("data/institute_scrutini.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["ord_scol","tipo_orario","num_anni","etichetta","percentuale_ammessi","anno","miur_code"])
    writer.writerows(institutes_ripetenti_data)


In [9]:
#id to search: accordionUl1

schools_link = []
with open("data/schoo_links.txt","r") as f:
    schools_link = f.readlines()



schools_endpoints = []
p1 = re.compile('var idteachingunit = (.*);')     
p2 = re.compile('var dateForfindtypeofcoursesofstudy = (.*);') 

for link in schools_link:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")

    scripts  = soup.find_all("script",{"scr":False})

    
    for script in scripts:
        if("idteachingunit" in script.text):
            match1 = p1.search(script.string)
            match2 = p2.search(script.string)

            teachingunit = match1.group(1)
            dateForfindtypeofcoursesofstudy = match2.group(1)
            schools_endpoints.append([teachingunit,dateForfindtypeofcoursesofstudy,link.split("/")[-1]])

    random.uniform(.9,1.4)


In [ ]:
# https://www.vivoscuola.it/seiimport/findtypeofcoursesofstudy/" + idteachingunit + "/" + dateForfindtypeofcoursesofstudy +"/"

school_courses = []
for endpoint in schools_endpoints:
    
    teachingunit, date, school_id = endpoint
    teachingunit = teachingunit[1:-1]
    date = date[1:-1]

    data_url = "https://www.vivoscuola.it/seiimport/findtypeofcoursesofstudy/"+teachingunit+"/"+date
    response = requests.get(url=data_url)
    data = response.json()
    print(endpoint)
    print(data_url)
    for indirizzo in data['accordionData']:
        title = indirizzo['Titolo']
        for articolazione in indirizzo['Item']:
            school_courses.append([teachingunit,date,title,articolazione,school_id])
    
    random.uniform(.9,1.4)
    

In [17]:
with open("data/school_courses.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["teaching_unit","date_for_find_type_of_courses_of_study","title","indirizzo","school_id"])
    writer.writerows(school_courses)